<a href="https://colab.research.google.com/github/sharathnallaganti/LLM-s/blob/main/S1_Model_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- System tools & OCR ---
!apt-get -y install tesseract-ocr poppler-utils > /dev/null

# --- Core Python libs ---
!pip install -q fastapi uvicorn pydantic==2.* \
    sentence-transformers faiss-cpu \
    pdfplumber pypdf python-docx unstructured[all-docs] \
    pillow rapidfuzz numpy pandas tqdm \
    httpx langchain==0.2.* langchain-community llama-index \
    pydantic-settings gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 63.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 132.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 119.4 MB/s eta 0:00:00
   ━━━

In [3]:
!pip install -q transformers accelerate bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "mistralai/Mistral-7B-Instruct-v0.2"  # small and fast enough
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype="auto"
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate(prompt):
    out = llm(prompt, max_new_tokens=200, temperature=0.2, do_sample=False)
    return out[0]["generated_text"]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
from sentence_transformers import SentenceTransformer
import faiss, numpy as np

# 1. Build small test index
texts = ["The Constitution was signed in 1787.",
         "Machine learning uses data to train models.",
         "Tesseract performs OCR on scanned documents."]
model = SentenceTransformer("all-MiniLM-L6-v2")

X = model.encode(texts, normalize_embeddings=True)
index = faiss.IndexFlatIP(X.shape[1])
index.add(np.array(X).astype("float32"))

# 2. Query
query = "When was the Constitution signed?"
q_emb = model.encode([query], normalize_embeddings=True).astype("float32")
scores, idx = index.search(q_emb, k=2)
context = "\n".join(texts[i] for i in idx[0])

# 3. Generate
prompt = f"Answer the question using the context.\n\nContext:\n{context}\n\nQuestion: {query}"
print(generate(prompt))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer the question using the context.

Context:
The Constitution was signed in 1787.
Tesseract performs OCR on scanned documents.

Question: When was the Constitution signed?
Answer: The Constitution was signed in 1787.

Explanation:
The context states that the Constitution was signed in 1787. The question asks for the same information, so the answer is taken directly from the context. Tesseract, a text recognition software, is not relevant to this question.


In [2]:
!pip install -q pdfplumber python-docx pypdf unstructured[all-docs]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.9 MB/s eta 0:00:00
   ━━━━

In [3]:
import os, json, re, pathlib, glob
import pdfplumber
from docx import Document

os.makedirs("/content/data_raw", exist_ok=True)
os.makedirs("/content/data_clean", exist_ok=True)

def clean(t):
    t = t.replace("\u00a0"," ").replace("\t"," ")
    t = re.sub(r"[ ]{2,}", " ", t)
    return t.strip()

out_path = "/content/data_clean/text.jsonl"
with open(out_path, "w", encoding="utf-8") as out:
    for path in glob.glob("/content/data_raw/**/*", recursive=True):
        if not os.path.isfile(path):
            continue
        ext = pathlib.Path(path).suffix.lower()
        doc_id = pathlib.Path(path).stem
        if ext == ".pdf":
            with pdfplumber.open(path) as pdf:
                for i, p in enumerate(pdf.pages, start=1):
                    txt = p.extract_text() or ""
                    if txt.strip():
                        rec = {"doc_id": doc_id, "page": i, "text": clean(txt), "meta": {"source": path}}
                        out.write(json.dumps(rec, ensure_ascii=False) + "\n")
        elif ext == ".docx":
            d = Document(path)
            txt = "\n".join(p.text for p in d.paragraphs)
            if txt.strip():
                rec = {"doc_id": doc_id, "page": None, "text": clean(txt), "meta": {"source": path}}
                out.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("Wrote:", out_path)


Wrote: /content/data_clean/text.jsonl


In [ ]:
from google.colab import files, drive
import os

os.makedirs("/content/data_raw", exist_ok=True)
uploaded = files.upload()  # select your PDFs/DOCXs
for name in uploaded.keys():
    !mv "{name}" /content/data_raw/
!ls -lh /content/data_raw | sed -n '1,50p'


In [ ]:
import os, json, re, pathlib, glob
import pdfplumber
from docx import Document

os.makedirs("/content/data_clean", exist_ok=True)

def clean(t):
    t = t.replace("\u00a0"," ").replace("\t"," ")
    t = re.sub(r"[ ]{2,}", " ", t)
    return t.strip()

out_path = "/content/data_clean/text.jsonl"
n_recs = 0

with open(out_path, "w", encoding="utf-8") as out:
    for path in sorted(glob.glob("/content/data_raw/**/*", recursive=True)):
        if not os.path.isfile(path):
            continue
        ext = pathlib.Path(path).suffix.lower()
        doc_id = pathlib.Path(path).stem
        if ext == ".pdf":
            try:
                with pdfplumber.open(path) as pdf:
                    for i, p in enumerate(pdf.pages, start=1):
                        txt = p.extract_text() or ""
                        if txt.strip():
                            rec = {"doc_id": doc_id, "page": i, "text": clean(txt), "meta": {"source": path}}
                            out.write(json.dumps(rec, ensure_ascii=False) + "\n")
                            n_recs += 1
            except Exception as e:
                print("PDF error:", path, e)
        elif ext == ".docx":
            try:
                d = Document(path)
                txt = "\n".join(par.text for par in d.paragraphs)
                if txt.strip():
                    rec = {"doc_id": doc_id, "page": None, "text": clean(txt), "meta": {"source": path}}
                    out.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    n_recs += 1
            except Exception as e:
                print("DOCX error:", path, e)

print("Wrote:", out_path, "| records:", n_recs)
!head -n 3 /content/data_clean/text.jsonl | sed -e 's/{/\n{/' -n '1,200p'
